# 探索性数据分析（EDA）


有趣的数据集，包含球员和裁判之间的故事！


数据集介绍点击： [here](https://osf.io/47tnc/).

<img src="figures/f1.png" alt="FAO" width="290" >

## 任务：

探索性数据分析（EDA）. 挑战目标: **这些裁判在给红牌的时候咋想的呢，会不会被跟球员的肤色有关?**


In [6]:
%matplotlib inline
%config InlineBackend.figure_format='retina'

#from __future__ import absolute_import, division, print_function
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.pyplot import GridSpec
import seaborn as sns
import numpy as np
import pandas as pd
import os, sys
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')
sns.set_context("poster", font_scale=1.3)

#import missingno as msno
#import pandas_profiling

from sklearn.datasets import make_blobs
import time

## 数据简介：

> 数据包含球员和裁判的信息，2012-2013年的比赛数据，总共设计球员2053名，裁判3147名，特征列表如下：

> -- https://docs.google.com/document/d/1uCF5wmbcL90qvrk_J27fWAvDcDNrO9o_APkicwRkOKc/edit


| Variable Name: | Variable Description: | 
| -- | -- | 
| playerShort | short player ID | 
| player | player name | 
| club | player club | 
| leagueCountry | country of player club (England, Germany, France, and Spain) | 
| height | player height (in cm) | 
| weight | player weight (in kg) | 
| position | player position | 
| games | number of games in the player-referee dyad | 
| goals | number of goals in the player-referee dyad | 
| yellowCards | number of yellow cards player received from the referee | 
| yellowReds | number of yellow-red cards player received from the referee | 
| redCards | number of red cards player received from the referee | 
| photoID | ID of player photo (if available) | 
| rater1 | skin rating of photo by rater 1 | 
| rater2 | skin rating of photo by rater 2 | 
| refNum | unique referee ID number (referee name removed for anonymizing purposes) | 
| refCountry | unique referee country ID number | 
| meanIAT | mean implicit bias score (using the race IAT) for referee country | 
| nIAT | sample size for race IAT in that particular country | 
| seIAT | standard error for mean estimate of race IAT   | 
| meanExp | mean explicit bias score (using a racial thermometer task) for referee country | 
| nExp | sample size for explicit bias in that particular country | 
| seExp |  standard error for mean estimate of explicit bias measure | 



In [7]:
# Uncomment one of the following lines and run the cell:

df = pd.read_csv("redcard.csv.gz", compression='gzip')


In [8]:
df.shape

(146028, 28)

In [9]:
df.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002


In [10]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
height,145765.0,181.935938,6.738726,1.610000e+02,177.000000,182.000000,187.000000,2.030000e+02
weight,143785.0,76.075662,7.140906,5.400000e+01,71.000000,76.000000,81.000000,1.000000e+02
games,146028.0,2.921166,3.413633,1.000000e+00,1.000000,2.000000,3.000000,4.700000e+01
victories,146028.0,1.278344,1.790725,0.000000e+00,0.000000,1.000000,2.000000,2.900000e+01
ties,146028.0,0.708241,1.116793,0.000000e+00,0.000000,0.000000,1.000000,1.400000e+01
defeats,146028.0,0.934581,1.383059,0.000000e+00,0.000000,1.000000,1.000000,1.800000e+01
goals,146028.0,0.338058,0.906481,0.000000e+00,0.000000,0.000000,0.000000,2.300000e+01
yellowCards,146028.0,0.385364,0.795333,0.000000e+00,0.000000,0.000000,1.000000,1.400000e+01
yellowReds,146028.0,0.011381,0.107931,0.000000e+00,0.000000,0.000000,0.000000,3.000000e+00
redCards,146028.0,0.012559,0.112889,0.000000e+00,0.000000,0.000000,0.000000,2.000000e+00


In [11]:
df.dtypes

playerShort       object
player            object
club              object
leagueCountry     object
birthday          object
height           float64
weight           float64
position          object
games              int64
victories          int64
ties               int64
defeats            int64
goals              int64
yellowCards        int64
yellowReds         int64
redCards           int64
photoID           object
rater1           float64
rater2           float64
refNum             int64
refCountry         int64
Alpha_3           object
meanIAT          float64
nIAT             float64
seIAT            float64
meanExp          float64
nExp             float64
seExp            float64
dtype: object

In [12]:
all_columns = df.columns.tolist()
all_columns

['playerShort',
 'player',
 'club',
 'leagueCountry',
 'birthday',
 'height',
 'weight',
 'position',
 'games',
 'victories',
 'ties',
 'defeats',
 'goals',
 'yellowCards',
 'yellowReds',
 'redCards',
 'photoID',
 'rater1',
 'rater2',
 'refNum',
 'refCountry',
 'Alpha_3',
 'meanIAT',
 'nIAT',
 'seIAT',
 'meanExp',
 'nExp',
 'seExp']

## Challenge

Before looking below, try to answer some high level questions about the dataset. 


How do we operationalize the question of referees giving more red cards to dark skinned players?
* Counterfactual: if the player were lighter, a ref is more likely to have given a yellow or no card **for the same offense under the same conditions**
* Regression: accounting for confounding, darker players have positive coefficient on regression against proportion red/total card

Potential issues
* How to combine rater1 and rater2? Average them? What if they disagree? Throw it out?
* Is data imbalanced, i.e. red cards are very rare?
* Is data biased, i.e. players have different amounts of play time? Is this a summary of their whole career?
* How do I know I've accounted for all forms of confounding?

**First, is there systematic discrimination across all refs?**

Exploration/hypotheses:
* Distribution of games played
* red cards vs games played
* Reds per game played vs total cards per game played by skin color
* Distribution of # red, # yellow, total cards, and fraction red per game played for all players by avg skin color
* How many refs did players encounter?
* Do some clubs play more aggresively and get carded more? Or are more reserved and get less?
* Does carding vary by leagueCountry?
* Do high scorers get more slack (fewer cards) for the same position?
* Are there some referees that give more red/yellow cards than others?
* how consistent are raters? Check with Cohen's kappa.
* how do red cards vary by position? e.g. defenders get more?
* Do players with more games get more cards, and is there difference across skin color?
* indication of bias depending on refCountry?

## Understand how the data's organized

The dataset is a single csv where it aggregated every interaction between referee and player into a single row. In other words: Referee A refereed Player B in, say, 10 games, and gave 2 redcards during those 10 games. Then there would be a unique row in the dataset that said: 

    Referee A, Player B, 2 redcards, ... 

This has several implications that make this first step to understanding and dealing with this data a bit tricky. First, is that the information about Player B is repeated each time -- meaning if we did a simple average of some metric of we would likely get a misleading result. 

For example, asking "what is the average `weight` of the players?"

In [13]:
df['height'].mean()

181.93593798236887

In [14]:
df['height'].mean()

181.93593798236887

In [15]:
df.groupby('playerShort').height.mean()

playerShort
aaron-hughes              182.0
aaron-hunt                183.0
aaron-lennon              165.0
aaron-ramsey              178.0
abdelhamid-el-kaoutari    180.0
abdon-prats               181.0
abdou-dampha              184.0
abdou-traore_2            180.0
abdoul-camara             177.0
abdoulaye-diallo_2        189.0
abdoulaye-diallo_3        182.0
abdoulaye-keita_2         188.0
abdoulaye-sane            184.0
abdoulwhaid-sissoko       180.0
abdul-rahman-baba         179.0
abdul-razak               180.0
abel-aguilar              185.0
abel-khaled               179.0
abelaziz-barrada          185.0
abou-diaby                188.0
aboulaye-keita            175.0
adam-bodzek               184.0
adam-campbell             168.0
adam-federici             188.0
adam-hlousek              188.0
adam-johnson              175.0
adam-lallana              173.0
adam-le-fondre            180.0
adam-morgan               179.0
adam-pinter               190.0
                          ..

In [16]:
np.mean(df.groupby('playerShort').height.mean())

181.74372848007872

## Tidy Data

Hadley Wickham's concept of a **tidy dataset** summarized as:

>  - Each variable forms a column
>  - Each observation forms a row
>  - Each type of observational unit forms a table

A longer paper describing this can be found in this [pdf](https://www.jstatsoft.org/article/view/v059i10/v59i10.pdf).

Having datasets in this form allows for much simpler analyses. So the first step is to try and clean up the dataset into a tidy dataset. 

The first step that I am going to take is to break up the dataset into the different observational units. By that I'm going to have separate tables (or dataframes) for: 

 - players
 - clubs
 - referees
 - countries
 - dyads

## Create Tidy Players Table

In [17]:
df2 = pd.DataFrame({'key1':['a', 'a', 'b', 'b', 'a'],
     'key2':['one', 'two', 'one', 'two', 'one'],
     'data1':np.random.randn(5),
     'data2':np.random.randn(5)})
df2

,key1,key2,data1,data2
0,a,one,-0.071263,-0.734116
1,a,two,0.882403,-1.598095
2,b,one,-0.031270,-1.497234
3,b,two,-1.118872,-0.206794
4,a,one,2.449043,1.059959


In [18]:
grouped = df2['data1'].groupby(df['key1'])
grouped.mean()

KeyError: 'key1'

In [19]:
player_index = 'playerShort'
player_cols = [#'player', # drop player name, we have unique identifier
               'birthday',
               'height',
               'weight',
               'position',
               'photoID',
               'rater1',
               'rater2',
              ]

In [20]:
all_cols_unique_players = df.groupby('playerShort')
all_cols_unique_players.head()

,playerShort,player,club,leagueCountry,birthday,height,weight,position,games,victories,...,rater2,refNum,refCountry,Alpha_3,meanIAT,nIAT,seIAT,meanExp,nExp,seExp
0,lucas-wilchez,Lucas Wilchez,Real Zaragoza,Spain,31.08.1983,177.0,72.0,Attacking Midfielder,1,0,...,0.50,1,1,GRC,0.326391,712.0,0.000564,0.396000,750.0,0.002696
1,john-utaka,John Utaka,Montpellier HSC,France,08.01.1982,179.0,82.0,Right Winger,1,0,...,0.75,2,2,ZMB,0.203375,40.0,0.010875,-0.204082,49.0,0.061504
2,abdon-prats,Abdón Prats,RCD Mallorca,Spain,17.12.1992,181.0,79.0,NaN,1,0,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
3,pablo-mari,Pablo Marí,RCD Mallorca,Spain,31.08.1993,191.0,87.0,Center Back,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
4,ruben-pena,Rubén Peña,Real Valladolid,Spain,18.07.1991,172.0,70.0,Right Midfielder,1,1,...,NaN,3,3,ESP,0.369894,1785.0,0.000229,0.588297,1897.0,0.001002
5,aaron-hughes,Aaron Hughes,Fulham FC,England,08.11.1979,182.0,71.0,Center Back,1,0,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
6,aleksandar-kolarov,Aleksandar Kolarov,Manchester City,England,10.11.1985,187.0,80.0,Left Fullback,1,1,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
7,alexander-tettey,Alexander Tettey,Norwich City,England,04.04.1986,180.0,68.0,Defensive Midfielder,1,0,...,1.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
8,anders-lindegaard,Anders Lindegaard,Manchester United,England,13.04.1984,193.0,80.0,Goalkeeper,1,0,...,0.25,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752
9,andreas-beck,Andreas Beck,1899 Hoffenheim,Germany,13.03.1987,180.0,70.0,Right Fullback,1,1,...,0.00,4,4,LUX,0.325185,127.0,0.003297,0.538462,130.0,0.013752


In [21]:
# Count the unique variables (if we got different weight values, 
# for example, then we should get more than one unique value in this groupby)
all_cols_unique_players = df.groupby('playerShort').agg({col:'nunique' for col in player_cols})

In [22]:
all_cols_unique_players.head()

,birthday,height,weight,position,photoID,rater1,rater2
playerShort,,,,,,,
aaron-hughes,1,1,1,1,1,1,1
aaron-hunt,1,1,1,1,1,1,1
aaron-lennon,1,1,1,1,1,1,1
aaron-ramsey,1,1,1,1,1,1,1
abdelhamid-el-kaoutari,1,1,1,1,1,1,1


In [31]:
# If all values are the same per player then this should be empty (and it is!)
all_cols_unique_players[all_cols_unique_players > 1].dropna().head()

,birthday,height,weight,position,photoID,rater1,rater2
playerShort,,,,,,,


In [24]:

# A slightly more elegant way to test the uniqueness
all_cols_unique_players[all_cols_unique_players > 1].dropna().shape[0] == 0

True

In [25]:
def get_subgroup(dataframe, g_index, g_columns):
    
    """Helper function that creates a sub-table from the columns and runs a quick uniqueness test."""
    g = dataframe.groupby(g_index).agg({col:'nunique' for col in g_columns})
    if g[g > 1].dropna().shape[0] != 0:
        print("Warning: you probably assumed this had all unique values but it doesn't.")
    return dataframe.groupby(g_index).agg({col:'max' for col in g_columns})

In [26]:
players = get_subgroup(df, player_index, player_cols)
players.head()

,birthday,height,weight,position,photoID,rater1,rater2
playerShort,,,,,,,
aaron-hughes,08.11.1979,182.0,71.0,Center Back,3868.jpg,0.25,0.00
aaron-hunt,04.09.1986,183.0,73.0,Attacking Midfielder,20136.jpg,0.00,0.25
aaron-lennon,16.04.1987,165.0,63.0,Right Midfielder,13515.jpg,0.25,0.25
aaron-ramsey,26.12.1990,178.0,76.0,Center Midfielder,94953.jpg,0.00,0.00
abdelhamid-el-kaoutari,17.03.1990,180.0,73.0,Center Back,124913.jpg,0.25,0.25


In [27]:
def save_subgroup(dataframe, g_index, subgroup_name, prefix='raw_'):
    save_subgroup_filename = "".join([prefix, subgroup_name, ".csv.gz"])
    dataframe.to_csv(save_subgroup_filename, compression='gzip', encoding='UTF-8')
    test_df = pd.read_csv(save_subgroup_filename, compression='gzip', index_col=g_index, encoding='UTF-8')
    # Test that we recover what we send in
    if dataframe.equals(test_df):
        print("Test-passed: we recover the equivalent subgroup dataframe.")
    else:
        print("Warning -- equivalence test!!! Double-check.")

In [28]:
players = get_subgroup(df, player_index, player_cols)
players.head()

,birthday,height,weight,position,photoID,rater1,rater2
playerShort,,,,,,,
aaron-hughes,08.11.1979,182.0,71.0,Center Back,3868.jpg,0.25,0.00
aaron-hunt,04.09.1986,183.0,73.0,Attacking Midfielder,20136.jpg,0.00,0.25
aaron-lennon,16.04.1987,165.0,63.0,Right Midfielder,13515.jpg,0.25,0.25
aaron-ramsey,26.12.1990,178.0,76.0,Center Midfielder,94953.jpg,0.00,0.00
abdelhamid-el-kaoutari,17.03.1990,180.0,73.0,Center Back,124913.jpg,0.25,0.25


In [29]:
save_subgroup(players, player_index, "players")

Test-passed: we recover the equivalent subgroup dataframe.


## Create Tidy Clubs Table

Create the clubs table.

In [32]:
club_index = 'club'
club_cols = ['leagueCountry']
clubs = get_subgroup(df, club_index, club_cols)
clubs.head()

,leagueCountry
club,
1. FC Nürnberg,Germany
1. FSV Mainz 05,Germany
1899 Hoffenheim,Germany
AC Ajaccio,France
AFC Bournemouth,England


In [33]:
clubs['leagueCountry'].value_counts()

England    48
Spain      27
France     22
Germany    21
Name: leagueCountry, dtype: int64

In [34]:
save_subgroup(clubs, club_index, "clubs", )

Test-passed: we recover the equivalent subgroup dataframe.


## Create Tidy Referees Table

In [35]:
referee_index = 'refNum'
referee_cols = ['refCountry']
referees = get_subgroup(df, referee_index, referee_cols)
referees.head()

,refCountry
refNum,
1,1
2,2
3,3
4,4
5,5


In [36]:
referees.refCountry.nunique()

161

In [37]:
referees.tail()

,refCountry
refNum,
3143,51
3144,55
3145,21
3146,51
3147,21


In [38]:
referees.shape

(3147, 1)

In [39]:
save_subgroup(referees, referee_index, "referees")

Test-passed: we recover the equivalent subgroup dataframe.


## Create Tidy Countries Table

In [41]:
country_index = 'refCountry'
country_cols = ['Alpha_3', # rename this name of country
                'meanIAT',
                'nIAT',
                'seIAT',
                'meanExp',
                'nExp',
                'seExp',
               ]
countries = get_subgroup(df, country_index, country_cols)
countries.head()

,Alpha_3,meanIAT,nIAT,meanExp,seExp,seIAT,nExp
refCountry,,,,,,,
1,GRC,0.326391,712.0,0.396000,0.002696,0.000564,750.0
2,ZMB,0.203375,40.0,-0.204082,0.061504,0.010875,49.0
3,ESP,0.369894,1785.0,0.588297,0.001002,0.000229,1897.0
4,LUX,0.325185,127.0,0.538462,0.013752,0.003297,130.0
5,TUN,0.167132,19.0,-0.789474,0.111757,0.027327,19.0


In [42]:
rename_columns = {'Alpha_3':'countryName', }
countries = countries.rename(columns=rename_columns)
countries.head()

,countryName,meanIAT,nIAT,meanExp,seExp,seIAT,nExp
refCountry,,,,,,,
1,GRC,0.326391,712.0,0.396000,0.002696,0.000564,750.0
2,ZMB,0.203375,40.0,-0.204082,0.061504,0.010875,49.0
3,ESP,0.369894,1785.0,0.588297,0.001002,0.000229,1897.0
4,LUX,0.325185,127.0,0.538462,0.013752,0.003297,130.0
5,TUN,0.167132,19.0,-0.789474,0.111757,0.027327,19.0


In [43]:
countries.shape

(161, 7)

In [44]:
save_subgroup(countries, country_index, "countries")

Warning -- equivalence test!!! Double-check.


## Create separate (not yet Tidy) Dyads Table

This is one of the more complex tables to reason about -- so we'll save it for a bit later. 

In [43]:
dyad_index = ['refNum', 'playerShort']
dyad_cols = ['games',
             'victories',
             'ties',
             'defeats',
             'goals',
             'yellowCards',
             'yellowReds',
             'redCards',
            ]

In [44]:
dyads = get_subgroup(df, g_index=dyad_index, g_columns=dyad_cols)

In [64]:
dyads.head()

redCards  victories  defeats  goals  games  yellowCards  \
refNum playerShort                                                              
1      lucas-wilchez         0          0        1      0      1            0   
2      john-utaka            0          0        1      0      1            1   
3      abdon-prats           0          0        0      0      1            1   
       pablo-mari            0          1        0      0      1            0   
       ruben-pena            0          1        0      0      1            0   

                      ties  yellowReds  
refNum playerShort                      
1      lucas-wilchez     0           0  
2      john-utaka        0           0  
3      abdon-prats       1           0  
       pablo-mari        0           0  
       ruben-pena        0           0

In [46]:
dyads.shape

(146028, 8)

In [47]:
dyads[dyads.redCards > 1].head(10)

yellowCards  yellowReds  victories  ties  games  \
refNum playerShort                                                          
140    bodipo                       0           0          2     1      6   
367    antonio-lopez_2              2           0          5     2      8   
432    javi-martinez                2           0          4     3     14   
       jonas                        0           0          1     4      9   
487    phil-jagielka                0           0          2     1      7   
586    cyril-jeunechamp             6           0          8     0     14   
804    sergio-ramos                 6           1         12     1     18   
985    aly-cissokho                 1           0          1     5      9   
1114   eugen-polanski               0           0          4     0      8   
1214   emmanuel-adebayor            4           1          9     7     23   

                          defeats  goals  redCards  
refNum playerShort                                  
140    bodipo                   3      1         2  
367    antonio-lopez_2          1      0         2  
432    javi-martinez            7      2         2  
       jonas                    4      1         2  
487    phil-jagielka            4      1         2  
586    cyril-jeunechamp         6      0         2  
804    sergio-ramos             5      4         2  
985    aly-cissokho             3      1         2  
1114   eugen-polanski           4      0         2  
1214   emmanuel-adebayor        7     10         2

In [48]:
save_subgroup(dyads, dyad_index, "dyads")

Test-passed: we recover the equivalent subgroup dataframe.


In [49]:
dyads.redCards.max()

2